In [2]:
import triton 
import torch
import triton.language as tl

In [15]:
@triton.jit 
def reverselist(inptr,outptr,m,blocksize:tl.constexpr):
    blockid=tl.program_id(0)
    block_start=blockid*blocksize
    offsets_a=block_start+tl.arange(0,blocksize)
    mask=offsets_a<m
    elem=tl.load(inptr+offsets_a,mask=mask)

    offsets_last=m-offsets_a-1
    mask_last=offsets_last>-1
    tl.store(outptr+offsets_last,elem,mask=mask_last)

In [16]:
def test():
   M=10
   a = torch.randn(M, device='cuda', dtype=torch.float16)
   b = torch.zeros_like(a)  
   
   blocksize = 128
   noofblocks = triton.cdiv(M, blocksize)  
   
   reverselist[(noofblocks,)](a, b, M,blocksize)  
   
   print(a)
   print(b)

In [17]:
if __name__=='__main__':
    test()

tensor([-1.1963, -0.1614, -1.0840,  0.8096, -1.9902, -1.0742,  0.3000, -0.4634,
        -0.7734,  0.4841], device='cuda:0', dtype=torch.float16)
tensor([ 0.4841, -0.7734, -0.4634,  0.3000, -1.0742, -1.9902,  0.8096, -1.0840,
        -0.1614, -1.1963], device='cuda:0', dtype=torch.float16)
